# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
vacation_df = pd.read_csv("../output_data/cities.csv")
vacation_df.dropna()
vacation_df.rename(columns={"Unnamed: 0":"City ID"}, inplace=True)
vacation_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ubinskoye,94,RU,1578416255,96,55.31,79.68,9.77,4.72
1,1,mahebourg,40,MU,1578416123,88,-20.41,57.70,80.60,3.36
2,2,yellowknife,20,CA,1578416255,73,62.45,-114.38,-32.01,4.70
3,3,jamestown,0,AU,1578416395,33,-33.21,138.60,70.74,14.79
4,4,khatanga,100,RU,1578416255,95,71.98,102.47,6.15,9.08
...,...,...,...,...,...,...,...,...,...,...
547,547,lisakovsk,99,KZ,1578416296,96,52.55,62.49,9.32,2.13
548,548,goderich,100,CA,1578416242,64,43.74,-81.71,39.99,4.00
549,549,mosquera,20,CO,1578416296,42,4.71,-74.23,64.40,6.93
550,550,wamba,77,CD,1578416216,56,2.14,27.99,74.97,2.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Use the Lat and Lng as locations and Humidity as the weight
locations =vacation_df[["Lat", "Lng"]]
weight = vacation_df["Humidity"].astype(float)

In [4]:
#Heat Map
fig = gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                 dissipating=False, max_intensity=100, 
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
weather_df=vacation_df.loc[(vacation_df["Max Temp"]<80)&(vacation_df["Max Temp"]>70)\
                           &(vacation_df["Wind Speed"]<10)&(vacation_df["Cloudiness"]==0)]
weather_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
415,415,acajutla,0,SV,1578416193,54,13.59,-89.83,78.8,5.82
476,476,mildura,0,AU,1578416291,46,-34.18,142.16,71.6,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

In [7]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,415,acajutla,0,SV,1578416193,54,13.59,-89.83,78.8,5.82,
1,476,mildura,0,AU,1578416291,46,-34.18,142.16,71.6,8.05,


In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_result = requests.get(url, params=params)
    hotel_result = hotel_result.json()
#     print(json.dumps(hotel_result, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_result["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing result...Skipping")
hotel_df
# print(hotel_result)

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,415,acajutla,0,SV,1578416193,54,13.59,-89.83,78.8,5.82,Villa Esmeralda
1,476,mildura,0,AU,1578416291,46,-34.18,142.16,71.6,8.05,Mercure Hotel Mildura


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, 
                             info_box_content=[f"Hotel loc:{rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))